# Converter CSV para SQL
Esse notebook lê os arquivos CSV e escreve um arquivo SQL que insere os dados no banco

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import pickle

In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
!pip install git+https://github.com/cinemagoer/cinemagoer
import imdb
ia = imdb.IMDb()

  Cloning https://github.com/cinemagoer/cinemagoer to /tmp/pip-req-build-80l5ognx
  Running command git clone -q https://github.com/cinemagoer/cinemagoer /tmp/pip-req-build-80l5ognx
  Created wheel for cinemagoer: filename=cinemagoer-2022.2.11-py3-none-any.whl size=301450 sha256=35dc659a32441fcf74d54fa7c99715a6befd4e455d000eb73ad201bcfb042025
  Stored in directory: /tmp/pip-ephem-wheel-cache-e11j303t/wheels/aa/25/77/eb71e4144eee1ee1883f440f5ec3898f09ab0291ed60193313
Successfully built cinemagoer


## 1. Leitura dos arquivos

Vamos ler os 4 arquivos em CSV, juntar todos em um único dataframe e remover a coluna 'show_id', pois o id no nosso banco não será o mesmo id do arquivo original

In [ ]:
# Le os 4 arquivos
amazon = pd.read_csv('amazon_prime_titles.csv')
disney = pd.read_csv('disney_plus_titles.csv')
hulu = pd.read_csv('hulu_titles.csv')
netflix = pd.read_csv('netflix_titles.csv')

# Adiciona one-hot de plataforma
amazon['amazon'] = True
disney['disney'] = True
netflix['netflix'] = True
hulu['hulu'] = True

# Adiciona data de inclusao para cada plataforma
amazon.rename(columns={'date_added': 'date_added_amazon'}, inplace=True)
disney.rename(columns={'date_added': 'date_added_disney'}, inplace=True)
hulu.rename(columns={'date_added': 'date_added_hulu'}, inplace=True)
netflix.rename(columns={'date_added': 'date_added_netflix'}, inplace=True)

# Junta em um dataframe so
shows = amazon.append(disney.append(hulu.append(netflix, ignore_index=True), ignore_index=True), ignore_index=True)

# Zera as outras plataformas
shows[['amazon','disney','hulu','netflix']] = shows[['amazon','disney','hulu','netflix']].fillna(False)

# Muda os vazios para NaN
shows.replace('', np.NaN)
 
# Remove a coluna 'show_id'
shows.drop(columns=['show_id'], inplace=True)

## 2. Limpeza de dados 

### 2.1: Conserta duração em lugar de classificação indicativa

In [ ]:
shows.loc[shows['rating'].str.contains(r"min|Season") == True,['duration']] = shows[shows['rating'].str.contains(r"min|Season") == True]['rating']
shows.loc[shows['rating'].str.contains(r"min|Season") == True,['rating']] = np.NaN

### 2.2: Junta os shows duplicados

In [ ]:
duplicatedShows = [v for v in shows.groupby(['title','type','release_year']).groups.values() if len(v)>1]

In [ ]:
def createSet(series):
  seriesSet = set()
  for seriesString in series.dropna().unique():
    for element in seriesString.split(', '):
      if(element != '' and element[-1] != ','): seriesSet.add(element)
  return seriesSet
    
def mergeRow(series):
  return ', '.join(createSet(series))

def dateAdded(dates):
  return np.NaN if dates.isnull().all() else dates.dropna().iloc[0]

mergedShows = {
  'type': [],
  'title': [],
  'release_year': [],
  'director': [],
  'cast': [],
  'country': [],
  'listed_in': [],
  'amazon': [],
  'disney': [],
  'hulu': [],
  'netflix': [],
  'date_added_amazon': [],
  'date_added_disney': [],
  'date_added_hulu': [],
  'date_added_netflix': [],
  'duration': [],
  'rating': [],
  'description': []
}
duplicatedIds = []

i = 0
n = len(duplicatedShows)

for showsId in duplicatedShows:
  duplicatedIds.extend(showsId)
  duplicatedPair = shows.loc[showsId]
  
  mergedShows['type'].append(duplicatedPair['type'].iloc[0])
  mergedShows['title'].append(duplicatedPair['title'].iloc[0])
  mergedShows['release_year'].append(duplicatedPair['release_year'].iloc[0])

  mergedShows['director'].append(mergeRow(duplicatedPair['director']))
  mergedShows['cast'].append(mergeRow(duplicatedPair['cast']))
  mergedShows['country'].append(mergeRow(duplicatedPair['country']))
  mergedShows['listed_in'].append(mergeRow(duplicatedPair['listed_in']))

  mergedShows['amazon'].append(duplicatedPair['amazon'].any())
  mergedShows['disney'].append(duplicatedPair['disney'].any())
  mergedShows['hulu'].append(duplicatedPair['hulu'].any())
  mergedShows['netflix'].append(duplicatedPair['netflix'].any())

  mergedShows['date_added_amazon'].append(dateAdded(duplicatedPair['date_added_amazon']))
  mergedShows['date_added_disney'].append(dateAdded(duplicatedPair['date_added_disney']))
  mergedShows['date_added_hulu'].append(dateAdded(duplicatedPair['date_added_hulu']))
  mergedShows['date_added_netflix'].append(dateAdded(duplicatedPair['date_added_netflix']))

  mergedShows['duration'].append(duplicatedPair.mode()['duration'].iloc[0])
  mergedShows['rating'].append(duplicatedPair.mode()['rating'].iloc[0])
  mergedShows['description'].append(duplicatedPair.mode()['description'].iloc[0])
  
  i += 1
  print('{:03.2f}%\r'.format(100*i/n),end='')
    
shows = shows.drop(duplicatedIds)
shows = shows.append(pd.DataFrame.from_dict(mergedShows), ignore_index=True)

### 2.3: Cria sets para países, gêneros, atores e diretores

In [ ]:
countries = pd.DataFrame(createSet(shows['country']), columns=['Name'])
genre = pd.DataFrame(createSet(shows['listed_in']), columns=['Name'])
actors = pd.DataFrame(createSet(shows['cast']), columns=['Name'])
directors = pd.DataFrame(createSet(shows['director']), columns=['Name'])

### 2.4: Obtenção de imagens

In [ ]:
count = 0
total = len(actors) + len(directors)
peoplePhoto = {}
with open('peoplePhoto.pickle','rb') as f:
  peoplePhoto = pickle.load(f)

def getActorFileName(name):
  if(name in peoplePhoto): return peoplePhoto[name]

  photoUrl = None
  try:
    person = ia.search_person(name)[0]
    photoUrl = person['full-size headshot']
  except KeyboardInterrupt:
    raise
  except:
    photoUrl = np.NaN
  
  peoplePhoto[name] = photoUrl
  with open('peoplePhoto.pickle','wb') as f:
    pickle.dump(peoplePhoto, f)
      
  return photoUrl
    
actors['photo'] = actors['Name'].progress_apply(getActorFileName)
directors['photo'] = directors['Name'].progress_apply(getActorFileName)

  0%|          | 0/62534 [00:00<?, ?it/s]

  0%|          | 0/10897 [00:00<?, ?it/s]

In [ ]:
actors = actors.replace('https://m.media-amazon.png', np.NaN)
directors = directors.replace('https://m.media-amazon.png', np.NaN)

### 2.5: Adição do código dos países

In [ ]:
countryCodeCSV = pd.read_csv('countries.csv')

# Vatican City = Holy See (Vatican City State)
countryCodeCSV.replace(['Holy See (Vatican City State)'],'Vatican City',inplace=True)

# Iran = Iran, Islamic Republic of
countryCodeCSV.replace(['Iran, Islamic Republic of'],'Iran',inplace=True)

# Namibia = Namibia ?????????????????
countryCodeCSV.replace(['Namibia'],'Namibia',inplace=True)

# Tanzania = Tanzania, United Republic of
countryCodeCSV.replace(['Tanzania, United Republic of'],'Tanzania',inplace=True)

# Palestine = Palestine, State of
countryCodeCSV.replace(['Palestine, State of'],'Palestine',inplace=True)

# Venezuela = Venezuela, Bolivarian Republic of
countryCodeCSV.replace(['Venezuela, Bolivarian Republic of'],'Venezuela',inplace=True)

# Syria = Syrian Arab Republic
countryCodeCSV.replace(['Syrian Arab Republic'],'Syria',inplace=True)

# Taiwan = Taiwan, Province of China
countryCodeCSV.replace(['Taiwan, Province of China'],'Taiwan',inplace=True)

# Russia = Russian Federation
countryCodeCSV.replace(['Russian Federation'],'Russia',inplace=True)

# South Korea = Korea, Republic of
countryCodeCSV.replace(['Korea, Republic of'],'South Korea',inplace=True)

# Vietnam = Viet Nam
countryCodeCSV.replace(['Viet Nam'],'Vietnam',inplace=True)

countries = countries.join(countryCodeCSV.set_index('Name'), on='Name')

## 3. Criação das tabelas

### 3.1: Separação dos dados

In [ ]:
Exhibit = {
  'id':[],
  'name':[],
  'release_year':[],
  'parental_rating':[],
  'description':[]
}

Country = {
  'country_code':[],
  'country_name':[],
  'id_exhibit':[]
}

Series = {
  'num_seasons':[],
  'id_exhibit':[]
}

Movie = {
  'runtime':[],
  'id_exhibit':[]
}

Actor = {
  'id':[],
  'name':[],
  'profile_photo':[]
}

Genre = {
  'id':[],
  'name':[]
}

Director = {
  'id':[],
  'name':[],
  'profile_photo':[]
}

Platform = {
  'id':[0,1,2,3],
  'name':['Amazon Prime', 'Netflix', 'Disney+', 'Hulu']
}

PlatformComposition = {
  'id_platform':[],
  'inclusion_date':[],
  'id_exhibit':[]
}

Directing = {
  'id_exhibit':[],
  'id_director':[]
}

GenreCategorization = {
  'id_exhibit':[],
  'id_genre':[]
}

Acting = {
  'id_exhibit':[],
  'id_actor':[]
}

In [ ]:
def populated_dict(show):
  id = show.name
  Exhibit['id'].append(id)
  Exhibit['name'].append(show['title'] if type(show['title']) != float else None)
  Exhibit['release_year'].append(show['release_year'] if type(show['release_year']) != float else None)
  Exhibit['parental_rating'].append(show['rating'] if type(show['rating']) != float else None)
  Exhibit['description'].append(show['description'] if type(show['description']) != float else None)

  if(type(show['country']) != float):
    for country in show['country'].split(', '):
      if(country == '' or country[-1] == ','): continue
      countryRow = countries[countries['Name'] == country]
      Country['country_code'].append(countryRow['Code'].iloc[0] if type(countryRow['Code']) != float else None)
      Country['country_name'].append(countryRow['Name'].iloc[0])
      Country['id_exhibit'].append(id)

  duration = show['duration']
  if(duration and type(duration) != float):
    durationValue = int(duration.split()[0])
    if('Season' in duration):
      Series['num_seasons'].append(durationValue)
      Series['id_exhibit'].append(id)
    else:
      Movie['runtime'].append(durationValue)
      Movie['id_exhibit'].append(id)

  if(show['amazon'] == 1):
    PlatformComposition['id_platform'].append(0)
    PlatformComposition['inclusion_date'].append(show['date_added_amazon'])
    PlatformComposition['id_exhibit'].append(id)

  if(show['netflix'] == 1):
    PlatformComposition['id_platform'].append(1)
    PlatformComposition['inclusion_date'].append(show['date_added_netflix'])
    PlatformComposition['id_exhibit'].append(id)

  if(show['disney'] == 1):
    PlatformComposition['id_platform'].append(2)
    PlatformComposition['inclusion_date'].append(show['date_added_disney'])
    PlatformComposition['id_exhibit'].append(id)

  if(show['hulu'] == 1):
    PlatformComposition['id_platform'].append(3)
    PlatformComposition['inclusion_date'].append(show['date_added_hulu'])
    PlatformComposition['id_exhibit'].append(id)
  
  if(type(show['director']) != float):
    for director in show['director'].split(', '):
      if(director == '' or director[-1] == ','): continue
      Directing['id_director'].append(directors[directors['Name'] == director].index[0])
      Directing['id_exhibit'].append(id)

  if(type(show['listed_in']) != float):
    for actual_genre in show['listed_in'].split(', '):
      if(actual_genre == '' or actual_genre[-1] == ','): continue
      GenreCategorization['id_genre'].append(genre[genre['Name'] == actual_genre].index[0])
      GenreCategorization['id_exhibit'].append(id)

  if(type(show['cast']) != float):
    for actor in show['cast'].split(', '):
      if(actor == '' or actor[-1] == ','): continue
      Acting['id_actor'].append(actors[actors['Name'] == actor].index[0])
      Acting['id_exhibit'].append(id)

shows.progress_apply(populated_dict, axis=1);

  0%|          | 0/22608 [00:00<?, ?it/s]

In [ ]:
Df_Exhibit = pd.DataFrame(Exhibit)
Df_Country = pd.DataFrame(Country)
Df_Series = pd.DataFrame(Series)
Df_Movie = pd.DataFrame(Movie)
Df_Actor = actors.reset_index()
Df_Genre = genre.reset_index()
Df_Director = directors.reset_index()
Df_Platform = pd.DataFrame(Platform)
Df_PlatformComposition = pd.DataFrame(PlatformComposition)
Df_Directing = pd.DataFrame(Directing)
Df_GenreCategorization = pd.DataFrame(GenreCategorization)
Df_Acting = pd.DataFrame(Acting)

### 3.2: Mudança de dados para o formato do modelo

In [ ]:
def format_date(date):
  if pd.isna(date): return date
  dateList = date.split()
  dateString = dateList[2] + '-'
  if(dateList[0] == 'January'): dateString += '01'
  if(dateList[0] == 'February'): dateString += '02'
  if(dateList[0] == 'March'): dateString += '03'
  if(dateList[0] == 'April'): dateString += '04'
  if(dateList[0] == 'May'): dateString += '05'
  if(dateList[0] == 'June'): dateString += '06'
  if(dateList[0] == 'July'): dateString += '07'
  if(dateList[0] == 'August'): dateString += '08'
  if(dateList[0] == 'September'): dateString += '09'
  if(dateList[0] == 'October'): dateString += '10'
  if(dateList[0] == 'November'): dateString += '11'
  if(dateList[0] == 'December'): dateString += '12'
  dateString += '-' + dateList[1][:-1]
  return dateString

Df_PlatformComposition['inclusion_date'] = Df_PlatformComposition['inclusion_date'].apply(format_date)

In [ ]:
Df_Country.loc[Df_Country['country_name'] == 'Kosovo', ['country_code']] = 'KO'
Df_Country.loc[Df_Country['country_name'] == 'Namibia', ['country_code']] = 'NA'
Df_Country.loc[Df_Country['country_name'] == 'West Germany', ['country_code']] = 'DD'
Df_Country.loc[Df_Country['country_name'] == 'Soviet Union', ['country_code']] = 'SU'
Df_Country.loc[Df_Country['country_name'] == 'East Germany', ['country_code']] = 'DW'

In [ ]:
Df_Acting.drop_duplicates(inplace = True)
Df_Directing.drop_duplicates(inplace = True)

### 3.3: Inserção no SQL

In [ ]:
def insert_df(data):
  insertString = ""
  for index, row in data.iterrows():
    insertString += '('
    for value in row.to_list():
      if(type(value) == str): insertString += "'"
      valueToAdd = value
      if(pd.isna(value)): valueToAdd = 'NULL'
      if(type(value) == str): valueToAdd = valueToAdd.replace("'", "''")
      if(type(value) == str): valueToAdd = valueToAdd.replace("\n", "")
      insertString += f"{valueToAdd}"
      if(type(value) == str): insertString += "'"
      insertString += ','
    insertString = insertString[:-1]
    insertString += '),\n'
  return insertString[:-2]

In [ ]:
with open('dump.sql','w') as f:
  f.write(f"insert into platforms values {insert_df(Df_Platform)};\n")
  f.write(f"insert into media values {insert_df(Df_Exhibit)};\n")
  f.write(f"insert into countries values {insert_df(Df_Country)};\n")
  f.write(f"insert into series values {insert_df(Df_Series)};\n")
  f.write(f"insert into movies values {insert_df(Df_Movie)};\n")
  f.write(f"insert into actors values {insert_df(Df_Actor)};\n")
  f.write(f"insert into genres values {insert_df(Df_Genre)};\n")
  f.write(f"insert into directors values {insert_df(Df_Director)};\n")
  f.write(f"insert into media_platform values {insert_df(Df_PlatformComposition)};\n")
  f.write(f"insert into director_media values {insert_df(Df_Directing)};\n")
  f.write(f"insert into genre_media values {insert_df(Df_GenreCategorization)};\n")
  f.write(f"insert into actor_media values {insert_df(Df_Acting)};\n")

## 4. Escrita e leitura dos dataframes
Rodar separadamente!!!

In [ ]:
with open('dataframes.pickle','wb') as f:
  pickle.dump([
           Df_Exhibit,
           Df_Country,
           Df_Series,
           Df_Movie,
           Df_Actor,
           Df_Genre,
           Df_Director,
           Df_Platform,
           Df_PlatformComposition,
           Df_Directing,
           Df_GenreCategorization,
           Df_Acting
          ], f)

In [ ]:
with open('dataframes.pickle','rb') as f:
  Df_Exhibit, Df_Country, Df_Series, Df_Movie, Df_Actor, Df_Genre, Df_Director, Df_Platform, Df_PlatformComposition, Df_Directing, Df_GenreCategorization, Df_Acting = pickle.load(f)